In [1]:
import nltk

# If haven't done so, go and download
# nltk.download()

In [2]:
import os

trend_files = [f.replace('.json', '') for f in os.listdir('../dataset/trends')]

In [3]:
import pandas as pd

tweet_df = pd.DataFrame([])

for i in trend_files:
    i = pd.read_json('../dataset/trends/'+i+'.json', orient='index')
    tweet_df = tweet_df.append(i).reset_index().drop('index', axis=1)

/Users/alvinjanuar/.pyenv/versions/3.9.2/lib/python3.9/site-packages/pandas/compat/__init__.py:124: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [4]:
tweet_df

,created_at,id,id_str,full_text,truncated,display_text_range,entities,metadata,source,in_reply_to_status_id,...,favorite_count,favorited,retweeted,lang,extended_entities,possibly_sensitive,quoted_status_id,quoted_status_id_str,inferred_text,quoted_status
0,2021-12-03 04:32:43+00:00,1466626426182930437,1466626426182930432,RT @yellowjimin_twt: REPLY AND RT WITH THE TAG...,False,"[0, 72]","{'hashtags': [{'text': 'AMAsTNT', 'indices': [...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://twitter.com/download/android"" ...",NaN,...,0,False,False,en,None,NaN,NaN,NaN,REPLY AND RT WITH THE TAGS 💜\n\n#AMAsTNT BTS @...,NaN
1,2021-12-03 04:02:46+00:00,1466618889446428680,1466618889446428672,RT @BTSBangtanLocks: Lockscreens|@BTS_twt \n#B...,False,"[0, 123]","{'hashtags': [{'text': 'BTSxAMAs', 'indices': ...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://twitter.com/download/android"" ...",NaN,...,0,False,False,en,"{'media': [{'id': 1462803892878647296, 'id_str...",0.0,NaN,NaN,Lockscreens|@BTS_twt \n#BTSxAMAs #AMAsTNT #BTS...,NaN
2,2021-12-03 02:47:43+00:00,1466600001597894658,1466600001597894656,RT @DBOYPICS: i love them 😭\n\n#AMAsTNT BTS @B...,False,"[0, 75]","{'hashtags': [{'text': 'AMAsTNT', 'indices': [...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://twitter.com/download/iphone"" r...",NaN,...,0,False,False,en,"{'media': [{'id': 1462618376157622273, 'id_str...",0.0,NaN,NaN,i love them 😭\n\n#AMAsTNT BTS @BTS_twt\n\nhttp...,NaN
3,2021-12-03 02:43:12+00:00,1466598865683894273,1466598865683894272,RT @ARMYTEAMIID: [📸]\n\nNew Kids on The Block ...,False,"[0, 140]","{'hashtags': [{'text': 'BTSxAMAs', 'indices': ...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://twitter.com/download/iphone"" r...",NaN,...,0,False,False,en,None,NaN,NaN,NaN,[📸]\n\nNew Kids on The Block bersama BTS.\n\n...,NaN
4,2021-12-03 02:42:27+00:00,1466598678701826053,1466598678701826048,RT @ARMYTEAMIID: [📸]\n\nShawn Stockman instagr...,False,"[0, 140]","{'hashtags': [{'text': 'BTSxAMAs', 'indices': ...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://twitter.com/download/iphone"" r...",NaN,...,0,False,False,en,None,0.0,NaN,NaN,[📸]\n\nShawn Stockman instagram update.\n\n📎 :...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16023,2021-12-03 05:43:57+00:00,1466644353602883586,1466644353602883584,Iiii mau liat cuplikan yang project seokjin ya...,False,"[0, 74]","{'hashtags': [], 'symbols': [], 'user_mentions...","{'iso_language_code': 'in', 'result_type': 're...","<a href=""http://twitter.com/download/android"" ...",NaN,...,0,False,False,in,None,NaN,NaN,NaN,Iiii mau liat cuplikan yang project seokjin ya...,None
16024,2021-12-03 05:43:57+00:00,1466644353590329345,1466644353590329344,SEOKJIN SAYANGKU 😭😭,False,"[0, 19]","{'hashtags': [], 'symbols': [], 'user_mentions...","{'iso_language_code': 'in', 'result_type': 're...","<a href=""http://twitter.com/download/iphone"" r...",NaN,...,0,False,False,in,None,NaN,NaN,NaN,SEOKJIN SAYANGKU 😭😭,None
16025,2021-12-03 05:43:57+00:00,1466644352428437504,1466644352428437504,RT @Lune412814: HAPPY JIN DAY\nKIM SEOKJIN\n#L...,False,"[0, 94]","{'hashtags': [{'text': 'LAHeartsJin', 'indices...","{'iso_language_code': 'in', 'result_type': 're...","<a href=""http://twitter.com/download/iphone"" r...",NaN,...,0,False,False,in,None,NaN,1.466644e+18,1.466644e+18,HAPPY JIN DAY\nKIM SEOKJIN\n#LAHeartsJin #Moon...,None
16026,2021-12-03 05:43:56+00:00,1466644351426068488,1466644351426068480,seokjin i love you😭,False,"[0, 19]","{'hashtags': [], 'symbols': [], 'user_mentions...","{'iso_language_code': 'in', 'result_type': 're...","<a href=""http://twitter.com/download/iphone"" r...",NaN,...,0,False,False,in,None,NaN,NaN,NaN,seokjin i love you😭,None


Goal: get sentiment. Therefore get only texts

In [5]:
tweets = pd.DataFrame([])
tweets['raw_text'] = tweet_df['inferred_text']
tweets.head()

,raw_text
0,REPLY AND RT WITH THE TAGS 💜\n\n#AMAsTNT BTS @...
1,Lockscreens|@BTS_twt \n#BTSxAMAs #AMAsTNT #BTS...
2,i love them 😭\n\n#AMAsTNT BTS @BTS_twt\n\nhttp...
3,[📸]\n\nNew Kids on The Block bersama BTS.\n\n...
4,[📸]\n\nShawn Stockman instagram update.\n\n📎 :...


In [6]:
tweets.shape

(16028, 1)

In [7]:
# 1st: clear Twitter-specific terms
import re

def remove_twitter(text, for_gensim = False):
    twitter_stripped_text = re.sub('(@[A-Za-z0-9_]+)', '', text) if for_gensim else re.sub('(@)', '', text)   # Username removal
    twitter_stripped_text = re.sub(r'([http(s)?):\/\/(www\.)?a-zA-Z0-9@:%._\+~#=]{2,256}\.[a-z]{2,6}\b([-a-zA-Z0-9@:%_\+.~#?&//=]*))', '', twitter_stripped_text)   # Link removal
    twitter_stripped_text = re.sub('(\#[A-Za-z0-9_]+)', '', twitter_stripped_text) if for_gensim else re.sub('(\#)', ' ', twitter_stripped_text)  # Hashtag removal, but preserving the word
    twitter_stripped_text = re.sub('([-&])', ' ', twitter_stripped_text)   # dash spacing, accommodating for better representation in spams
    return twitter_stripped_text

tweets['1st_clean_twitter'] = tweets['raw_text'].apply(lambda x: remove_twitter(x))
tweets['1st_clean_twitter_gensim'] = tweets['raw_text'].apply(lambda x: remove_twitter(x, for_gensim=True))
tweets.head()

,raw_text,1st_clean_twitter,1st_clean_twitter_gensim
0,REPLY AND RT WITH THE TAGS 💜\n\n#AMAsTNT BTS @...,REPLY AND RT WITH THE TAGS 💜\n\n AMAsTNT BTS B...,REPLY AND RT WITH THE TAGS 💜\n\n BTS
1,Lockscreens|@BTS_twt \n#BTSxAMAs #AMAsTNT #BTS...,Lockscreens|BTS_twt \n BTSxAMAs AMAsTNT BTS\...,Lockscreens| \n \nDon't repost! be honest!∅ \n
2,i love them 😭\n\n#AMAsTNT BTS @BTS_twt\n\nhttp...,i love them 😭\n\n AMAsTNT BTS BTS_twt\n\n,i love them 😭\n\n BTS \n\n
3,[📸]\n\nNew Kids on The Block bersama BTS.\n\n...,[📸]\n\nNew Kids on The Block bersama BTS.\n\n...,[📸]\n\nNew Kids on The Block bersama BTS.\n\n...
4,[📸]\n\nShawn Stockman instagram update.\n\n📎 :...,[📸]\n\nShawn Stockman instagram update.\n\n📎 :...,[📸]\n\nShawn Stockman instagram update.\n\n📎 :...


In [8]:
# 2nd: remove punctuations
import string
punctuation_gensim = re.sub('([.,?!])','', string.punctuation)

def remove_punctuation(text, for_gensim=False):
    clean_text = "".join([i for i in text if i not in punctuation_gensim]) if for_gensim else "".join([i for i in text if i not in string.punctuation])
    clean_text = re.sub('([’])', "'", clean_text)
    return clean_text

tweets['2nd_punctuation'] = tweets['1st_clean_twitter'].apply(lambda x: remove_punctuation(x))
tweets['2nd_punctuation_gensim'] = tweets['1st_clean_twitter_gensim'].apply(lambda x: remove_punctuation(x, for_gensim=True))
tweets.head()

,raw_text,1st_clean_twitter,1st_clean_twitter_gensim,2nd_punctuation,2nd_punctuation_gensim
0,REPLY AND RT WITH THE TAGS 💜\n\n#AMAsTNT BTS @...,REPLY AND RT WITH THE TAGS 💜\n\n AMAsTNT BTS B...,REPLY AND RT WITH THE TAGS 💜\n\n BTS,REPLY AND RT WITH THE TAGS 💜\n\n AMAsTNT BTS B...,REPLY AND RT WITH THE TAGS 💜\n\n BTS
1,Lockscreens|@BTS_twt \n#BTSxAMAs #AMAsTNT #BTS...,Lockscreens|BTS_twt \n BTSxAMAs AMAsTNT BTS\...,Lockscreens| \n \nDon't repost! be honest!∅ \n,LockscreensBTStwt \n BTSxAMAs AMAsTNT BTS\nD...,Lockscreens \n \nDont repost! be honest!∅ \n
2,i love them 😭\n\n#AMAsTNT BTS @BTS_twt\n\nhttp...,i love them 😭\n\n AMAsTNT BTS BTS_twt\n\n,i love them 😭\n\n BTS \n\n,i love them 😭\n\n AMAsTNT BTS BTStwt\n\n,i love them 😭\n\n BTS \n\n
3,[📸]\n\nNew Kids on The Block bersama BTS.\n\n...,[📸]\n\nNew Kids on The Block bersama BTS.\n\n...,[📸]\n\nNew Kids on The Block bersama BTS.\n\n...,📸\n\nNew Kids on The Block bersama BTS\n\nDon...,📸\n\nNew Kids on The Block bersama BTS.\n\nDo...
4,[📸]\n\nShawn Stockman instagram update.\n\n📎 :...,[📸]\n\nShawn Stockman instagram update.\n\n📎 :...,[📸]\n\nShawn Stockman instagram update.\n\n📎 :...,📸\n\nShawn Stockman instagram update\n\n📎 \n...,📸\n\nShawn Stockman instagram update.\n\n📎 \...


In [9]:
import emoji

def remove_emoji(text):
    return re.sub(emoji.get_emoji_regexp(), r"", text)

tweets['3rd_emoji'] = tweets['2nd_punctuation'].apply(lambda x: remove_emoji(x))
tweets['3rd_emoji_gensim'] = tweets['2nd_punctuation_gensim'].apply(lambda x: remove_emoji(x))
tweets.head()

,raw_text,1st_clean_twitter,1st_clean_twitter_gensim,2nd_punctuation,2nd_punctuation_gensim,3rd_emoji,3rd_emoji_gensim
0,REPLY AND RT WITH THE TAGS 💜\n\n#AMAsTNT BTS @...,REPLY AND RT WITH THE TAGS 💜\n\n AMAsTNT BTS B...,REPLY AND RT WITH THE TAGS 💜\n\n BTS,REPLY AND RT WITH THE TAGS 💜\n\n AMAsTNT BTS B...,REPLY AND RT WITH THE TAGS 💜\n\n BTS,REPLY AND RT WITH THE TAGS \n\n AMAsTNT BTS BT...,REPLY AND RT WITH THE TAGS \n\n BTS
1,Lockscreens|@BTS_twt \n#BTSxAMAs #AMAsTNT #BTS...,Lockscreens|BTS_twt \n BTSxAMAs AMAsTNT BTS\...,Lockscreens| \n \nDon't repost! be honest!∅ \n,LockscreensBTStwt \n BTSxAMAs AMAsTNT BTS\nD...,Lockscreens \n \nDont repost! be honest!∅ \n,LockscreensBTStwt \n BTSxAMAs AMAsTNT BTS\nD...,Lockscreens \n \nDont repost! be honest!∅ \n
2,i love them 😭\n\n#AMAsTNT BTS @BTS_twt\n\nhttp...,i love them 😭\n\n AMAsTNT BTS BTS_twt\n\n,i love them 😭\n\n BTS \n\n,i love them 😭\n\n AMAsTNT BTS BTStwt\n\n,i love them 😭\n\n BTS \n\n,i love them \n\n AMAsTNT BTS BTStwt\n\n,i love them \n\n BTS \n\n
3,[📸]\n\nNew Kids on The Block bersama BTS.\n\n...,[📸]\n\nNew Kids on The Block bersama BTS.\n\n...,[📸]\n\nNew Kids on The Block bersama BTS.\n\n...,📸\n\nNew Kids on The Block bersama BTS\n\nDon...,📸\n\nNew Kids on The Block bersama BTS.\n\nDo...,\n\nNew Kids on The Block bersama BTS\n\nDonn...,\n\nNew Kids on The Block bersama BTS.\n\nDon...
4,[📸]\n\nShawn Stockman instagram update.\n\n📎 :...,[📸]\n\nShawn Stockman instagram update.\n\n📎 :...,[📸]\n\nShawn Stockman instagram update.\n\n📎 :...,📸\n\nShawn Stockman instagram update\n\n📎 \n...,📸\n\nShawn Stockman instagram update.\n\n📎 \...,\n\nShawn Stockman instagram update\n\n \n\n...,\n\nShawn Stockman instagram update.\n\n \n\...


In [10]:
tweets['4th_tokenized'] = tweets['3rd_emoji'].apply(nltk.tokenize.word_tokenize)
tweets['4th_tokenized_gensim'] = tweets['3rd_emoji_gensim'].apply(nltk.tokenize.word_tokenize)
tweets.head()

,raw_text,1st_clean_twitter,1st_clean_twitter_gensim,2nd_punctuation,2nd_punctuation_gensim,3rd_emoji,3rd_emoji_gensim,4th_tokenized,4th_tokenized_gensim
0,REPLY AND RT WITH THE TAGS 💜\n\n#AMAsTNT BTS @...,REPLY AND RT WITH THE TAGS 💜\n\n AMAsTNT BTS B...,REPLY AND RT WITH THE TAGS 💜\n\n BTS,REPLY AND RT WITH THE TAGS 💜\n\n AMAsTNT BTS B...,REPLY AND RT WITH THE TAGS 💜\n\n BTS,REPLY AND RT WITH THE TAGS \n\n AMAsTNT BTS BT...,REPLY AND RT WITH THE TAGS \n\n BTS,"[REPLY, AND, RT, WITH, THE, TAGS, AMAsTNT, BTS...","[REPLY, AND, RT, WITH, THE, TAGS, BTS]"
1,Lockscreens|@BTS_twt \n#BTSxAMAs #AMAsTNT #BTS...,Lockscreens|BTS_twt \n BTSxAMAs AMAsTNT BTS\...,Lockscreens| \n \nDon't repost! be honest!∅ \n,LockscreensBTStwt \n BTSxAMAs AMAsTNT BTS\nD...,Lockscreens \n \nDont repost! be honest!∅ \n,LockscreensBTStwt \n BTSxAMAs AMAsTNT BTS\nD...,Lockscreens \n \nDont repost! be honest!∅ \n,"[LockscreensBTStwt, BTSxAMAs, AMAsTNT, BTS, Do...","[Lockscreens, Dont, repost, !, be, honest, !, ∅]"
2,i love them 😭\n\n#AMAsTNT BTS @BTS_twt\n\nhttp...,i love them 😭\n\n AMAsTNT BTS BTS_twt\n\n,i love them 😭\n\n BTS \n\n,i love them 😭\n\n AMAsTNT BTS BTStwt\n\n,i love them 😭\n\n BTS \n\n,i love them \n\n AMAsTNT BTS BTStwt\n\n,i love them \n\n BTS \n\n,"[i, love, them, AMAsTNT, BTS, BTStwt]","[i, love, them, BTS]"
3,[📸]\n\nNew Kids on The Block bersama BTS.\n\n...,[📸]\n\nNew Kids on The Block bersama BTS.\n\n...,[📸]\n\nNew Kids on The Block bersama BTS.\n\n...,📸\n\nNew Kids on The Block bersama BTS\n\nDon...,📸\n\nNew Kids on The Block bersama BTS.\n\nDo...,\n\nNew Kids on The Block bersama BTS\n\nDonn...,\n\nNew Kids on The Block bersama BTS.\n\nDon...,"[New, Kids, on, The, Block, bersama, BTS, Donn...","[New, Kids, on, The, Block, bersama, BTS, ., D..."
4,[📸]\n\nShawn Stockman instagram update.\n\n📎 :...,[📸]\n\nShawn Stockman instagram update.\n\n📎 :...,[📸]\n\nShawn Stockman instagram update.\n\n📎 :...,📸\n\nShawn Stockman instagram update\n\n📎 \n...,📸\n\nShawn Stockman instagram update.\n\n📎 \...,\n\nShawn Stockman instagram update\n\n \n\n...,\n\nShawn Stockman instagram update.\n\n \n\...,"[Shawn, Stockman, instagram, update, BTSxAMAs,...","[Shawn, Stockman, instagram, update, ., 방탄소년단]"


In [11]:
# On the other project that I made I chose stopwords because I was going for bag-of-words approach.
crf = nltk.CRFTagger()
crf.set_model_file('../resources/pos_tag/all_indo_man_tag_corpus_model.crf.tagger')

_ = crf.tag_sents(tweets['4th_tokenized'])

In [12]:
_

[[('REPLY', 'NNP'),
  ('AND', 'NNP'),
  ('RT', 'NNP'),
  ('WITH', 'NNP'),
  ('THE', 'NNP'),
  ('TAGS', 'NNP'),
  ('AMAsTNT', 'NNP'),
  ('BTS', 'NNP'),
  ('BTStwt', 'NNP')],
 [('LockscreensBTStwt', 'NNP'),
  ('BTSxAMAs', 'NNP'),
  ('AMAsTNT', 'NNP'),
  ('BTS', 'FW'),
  ('Dont', 'FW'),
  ('repost', 'FW'),
  ('be', 'FW'),
  ('honest∅', 'FW'),
  ('vica', 'FW')],
 [('i', 'FW'),
  ('love', 'FW'),
  ('them', 'FW'),
  ('AMAsTNT', 'FW'),
  ('BTS', 'FW'),
  ('BTStwt', 'FW')],
 [('New', 'NNP'),
  ('Kids', 'NNP'),
  ('on', 'NNP'),
  ('The', 'NNP'),
  ('Block', 'NNP'),
  ('bersama', 'IN'),
  ('BTS', 'NNP'),
  ('Donnie', 'NNP'),
  ('Wahlberg', 'NNP'),
  ('menggunakan', 'VB'),
  ('T', 'NNP'),
  ('shirt', 'FW'),
  ('BTS', 'FW'),
  ('Love', 'FW'),
  ('yourself', 'FW'),
  ('world', 'FW'),
  ('tour', 'FW'),
  ('BTSxAMAs', 'NNP'),
  ('AMAsTNT', 'NNP'),
  ('방탄소년단', 'NNP'),
  ('BTS', 'NNP'),
  ('BTStwt', 'NNP')],
 [('Shawn', 'NNP'),
  ('Stockman', 'NNP'),
  ('instagram', 'FW'),
  ('update', 'FW'),
  ('BTSxA

From here we can see that the POS tagger we use did not tag a bit of the text correctly. Another solution that we have is to load an established Indonesian corpus (from Flair), train the model ourselves, and use it as our POS tagger.

In [13]:
from flair.models import SequenceTagger

pos_tag = SequenceTagger.load('../resources/taggers/indonesian-pos/final-model.pt')

2021-12-10 16:25:26,183 loading file ../resources/taggers/indonesian-pos/final-model.pt


In [14]:
from flair.data import Sentence

Sentence('Budi pergi ke pasar')
# pos_tag.predict(tweets['4th_tokenized'].iloc[0])

Sentence: "Budi pergi ke pasar"   [− Tokens: 4]

In [15]:
tweets['sentence'] = tweets['4th_tokenized'].apply(Sentence)
tweets['sentence'].head()

0    (Token: 1 REPLY, Token: 2 AND, Token: 3 RT, To...
1    (Token: 1 LockscreensBTStwt, Token: 2 BTSxAMAs...
2    (Token: 1 i, Token: 2 love, Token: 3 them, Tok...
3    (Token: 1 New, Token: 2 Kids, Token: 3 on, Tok...
4    (Token: 1 Shawn, Token: 2 Stockman, Token: 3 i...
Name: sentence, dtype: object

In [16]:
tweets['sentence']

0        (Token: 1 REPLY, Token: 2 AND, Token: 3 RT, To...
1        (Token: 1 LockscreensBTStwt, Token: 2 BTSxAMAs...
2        (Token: 1 i, Token: 2 love, Token: 3 them, Tok...
3        (Token: 1 New, Token: 2 Kids, Token: 3 on, Tok...
4        (Token: 1 Shawn, Token: 2 Stockman, Token: 3 i...
                               ...                        
16023    (Token: 1 Iiii, Token: 2 mau, Token: 3 liat, T...
16024                (Token: 1 SEOKJIN, Token: 2 SAYANGKU)
16025    (Token: 1 HAPPY, Token: 2 JIN, Token: 3 DAY, T...
16026    (Token: 1 seokjin, Token: 2 i, Token: 3 love, ...
16027    (Token: 1 Surprise, Token: 2 nya, Token: 3 LA,...
Name: sentence, Length: 16028, dtype: object

In [17]:
tweets['sentence'].apply(pos_tag.predict)

0        None
1        None
2        None
3        None
4        None
         ... 
16023    None
16024    None
16025    None
16026    None
16027    None
Name: sentence, Length: 16028, dtype: object

In [18]:
tweets['sentence']

0        (Token: 1 REPLY, Token: 2 AND, Token: 3 RT, To...
1        (Token: 1 LockscreensBTStwt, Token: 2 BTSxAMAs...
2        (Token: 1 i, Token: 2 love, Token: 3 them, Tok...
3        (Token: 1 New, Token: 2 Kids, Token: 3 on, Tok...
4        (Token: 1 Shawn, Token: 2 Stockman, Token: 3 i...
                               ...                        
16023    (Token: 1 Iiii, Token: 2 mau, Token: 3 liat, T...
16024                (Token: 1 SEOKJIN, Token: 2 SAYANGKU)
16025    (Token: 1 HAPPY, Token: 2 JIN, Token: 3 DAY, T...
16026    (Token: 1 seokjin, Token: 2 i, Token: 3 love, ...
16027    (Token: 1 Surprise, Token: 2 nya, Token: 3 LA,...
Name: sentence, Length: 16028, dtype: object

In [19]:
_ = tweets['sentence'].iloc[1]
_.to_tagged_string()

'LockscreensBTStwt <PROPN> BTSxAMAs <PROPN> AMAsTNT <PROPN> BTS <PROPN> Dont <PROPN> repost <PROPN> be <PROPN> honest∅ <PROPN> vica <PROPN>'

In [20]:
re.findall(r'(<[^>]*>)', _.to_tagged_string())

['<PROPN>',
 '<PROPN>',
 '<PROPN>',
 '<PROPN>',
 '<PROPN>',
 '<PROPN>',
 '<PROPN>',
 '<PROPN>',
 '<PROPN>']

In [21]:
# Trial

re.sub('[ ]<', '_', _.to_tagged_string()).replace('>', '')

'LockscreensBTStwt_PROPN BTSxAMAs_PROPN AMAsTNT_PROPN BTS_PROPN Dont_PROPN repost_PROPN be_PROPN honest∅_PROPN vica_PROPN'

In [22]:
sentence_tagged = []

for i in tweets['sentence']:
    sentence_tagged.append(re.sub(r'[ ]<', '_', i.to_tagged_string()).replace('>', ''))

sentence_tagged

['REPLY_PROPN AND_PROPN RT_PROPN WITH_PROPN THE_PROPN TAGS_PROPN AMAsTNT_PROPN BTS_PROPN BTStwt_PUNCT',
 'LockscreensBTStwt_PROPN BTSxAMAs_PROPN AMAsTNT_PROPN BTS_PROPN Dont_PROPN repost_PROPN be_PROPN honest∅_PROPN vica_PROPN',
 'i_PROPN love_PROPN them_PROPN AMAsTNT_PROPN BTS_PROPN BTStwt_PUNCT',
 'New_PROPN Kids_PROPN on_PROPN The_PROPN Block_PROPN bersama_ADP BTS_PROPN Donnie_PROPN Wahlberg_PROPN menggunakan_VERB T_PROPN shirt_PROPN BTS_PROPN Love_PROPN yourself_PROPN world_PROPN tour_PROPN BTSxAMAs_PROPN AMAsTNT_PROPN 방탄소년단_PROPN BTS_PROPN BTStwt_PUNCT',
 'Shawn_PROPN Stockman_PROPN instagram_PROPN update_PROPN BTSxAMAs_PROPN AMAsTNT_PROPN 방탄소년단_PROPN BTS_PROPN BTStwt_PUNCT',
 'JUNGKOOK_PROPN AMAs_PROPN HD_PROPN WALLPAPER_PROPN amp_PROPN LOCKSCREEN_PROPN reply_PROPN and_PROPN retweet_PROPN AMAsTNT_PROPN BTS_PROPN BTStwt_PROPN BTS_PROPN BTSxAMAs_PROPN 방탄소년단_PROPN BTSWALLPAPER_PROPN JUNGKOOK_PROPN btslockscreen_PROPN 방탄배경_PUNCT',
 'hobi_NOUN and_PROPN becky_PROPN AMAsTNT_PROPN BTS_P

In [23]:
sentence_tagged

['REPLY_PROPN AND_PROPN RT_PROPN WITH_PROPN THE_PROPN TAGS_PROPN AMAsTNT_PROPN BTS_PROPN BTStwt_PUNCT',
 'LockscreensBTStwt_PROPN BTSxAMAs_PROPN AMAsTNT_PROPN BTS_PROPN Dont_PROPN repost_PROPN be_PROPN honest∅_PROPN vica_PROPN',
 'i_PROPN love_PROPN them_PROPN AMAsTNT_PROPN BTS_PROPN BTStwt_PUNCT',
 'New_PROPN Kids_PROPN on_PROPN The_PROPN Block_PROPN bersama_ADP BTS_PROPN Donnie_PROPN Wahlberg_PROPN menggunakan_VERB T_PROPN shirt_PROPN BTS_PROPN Love_PROPN yourself_PROPN world_PROPN tour_PROPN BTSxAMAs_PROPN AMAsTNT_PROPN 방탄소년단_PROPN BTS_PROPN BTStwt_PUNCT',
 'Shawn_PROPN Stockman_PROPN instagram_PROPN update_PROPN BTSxAMAs_PROPN AMAsTNT_PROPN 방탄소년단_PROPN BTS_PROPN BTStwt_PUNCT',
 'JUNGKOOK_PROPN AMAs_PROPN HD_PROPN WALLPAPER_PROPN amp_PROPN LOCKSCREEN_PROPN reply_PROPN and_PROPN retweet_PROPN AMAsTNT_PROPN BTS_PROPN BTStwt_PROPN BTS_PROPN BTSxAMAs_PROPN 방탄소년단_PROPN BTSWALLPAPER_PROPN JUNGKOOK_PROPN btslockscreen_PROPN 방탄배경_PUNCT',
 'hobi_NOUN and_PROPN becky_PROPN AMAsTNT_PROPN BTS_P

In [24]:
tweets['5_tagged'] = sentence_tagged
tweets['5_tagged'] = tweets['5_tagged'].apply(nltk.tokenize.word_tokenize)

In [25]:
tweets.head()

,raw_text,1st_clean_twitter,1st_clean_twitter_gensim,2nd_punctuation,2nd_punctuation_gensim,3rd_emoji,3rd_emoji_gensim,4th_tokenized,4th_tokenized_gensim,sentence,5_tagged
0,REPLY AND RT WITH THE TAGS 💜\n\n#AMAsTNT BTS @...,REPLY AND RT WITH THE TAGS 💜\n\n AMAsTNT BTS B...,REPLY AND RT WITH THE TAGS 💜\n\n BTS,REPLY AND RT WITH THE TAGS 💜\n\n AMAsTNT BTS B...,REPLY AND RT WITH THE TAGS 💜\n\n BTS,REPLY AND RT WITH THE TAGS \n\n AMAsTNT BTS BT...,REPLY AND RT WITH THE TAGS \n\n BTS,"[REPLY, AND, RT, WITH, THE, TAGS, AMAsTNT, BTS...","[REPLY, AND, RT, WITH, THE, TAGS, BTS]","(Token: 1 REPLY, Token: 2 AND, Token: 3 RT, To...","[REPLY_PROPN, AND_PROPN, RT_PROPN, WITH_PROPN,..."
1,Lockscreens|@BTS_twt \n#BTSxAMAs #AMAsTNT #BTS...,Lockscreens|BTS_twt \n BTSxAMAs AMAsTNT BTS\...,Lockscreens| \n \nDon't repost! be honest!∅ \n,LockscreensBTStwt \n BTSxAMAs AMAsTNT BTS\nD...,Lockscreens \n \nDont repost! be honest!∅ \n,LockscreensBTStwt \n BTSxAMAs AMAsTNT BTS\nD...,Lockscreens \n \nDont repost! be honest!∅ \n,"[LockscreensBTStwt, BTSxAMAs, AMAsTNT, BTS, Do...","[Lockscreens, Dont, repost, !, be, honest, !, ∅]","(Token: 1 LockscreensBTStwt, Token: 2 BTSxAMAs...","[LockscreensBTStwt_PROPN, BTSxAMAs_PROPN, AMAs..."
2,i love them 😭\n\n#AMAsTNT BTS @BTS_twt\n\nhttp...,i love them 😭\n\n AMAsTNT BTS BTS_twt\n\n,i love them 😭\n\n BTS \n\n,i love them 😭\n\n AMAsTNT BTS BTStwt\n\n,i love them 😭\n\n BTS \n\n,i love them \n\n AMAsTNT BTS BTStwt\n\n,i love them \n\n BTS \n\n,"[i, love, them, AMAsTNT, BTS, BTStwt]","[i, love, them, BTS]","(Token: 1 i, Token: 2 love, Token: 3 them, Tok...","[i_PROPN, love_PROPN, them_PROPN, AMAsTNT_PROP..."
3,[📸]\n\nNew Kids on The Block bersama BTS.\n\n...,[📸]\n\nNew Kids on The Block bersama BTS.\n\n...,[📸]\n\nNew Kids on The Block bersama BTS.\n\n...,📸\n\nNew Kids on The Block bersama BTS\n\nDon...,📸\n\nNew Kids on The Block bersama BTS.\n\nDo...,\n\nNew Kids on The Block bersama BTS\n\nDonn...,\n\nNew Kids on The Block bersama BTS.\n\nDon...,"[New, Kids, on, The, Block, bersama, BTS, Donn...","[New, Kids, on, The, Block, bersama, BTS, ., D...","(Token: 1 New, Token: 2 Kids, Token: 3 on, Tok...","[New_PROPN, Kids_PROPN, on_PROPN, The_PROPN, B..."
4,[📸]\n\nShawn Stockman instagram update.\n\n📎 :...,[📸]\n\nShawn Stockman instagram update.\n\n📎 :...,[📸]\n\nShawn Stockman instagram update.\n\n📎 :...,📸\n\nShawn Stockman instagram update\n\n📎 \n...,📸\n\nShawn Stockman instagram update.\n\n📎 \...,\n\nShawn Stockman instagram update\n\n \n\n...,\n\nShawn Stockman instagram update.\n\n \n\...,"[Shawn, Stockman, instagram, update, BTSxAMAs,...","[Shawn, Stockman, instagram, update, ., 방탄소년단]","(Token: 1 Shawn, Token: 2 Stockman, Token: 3 i...","[Shawn_PROPN, Stockman_PROPN, instagram_PROPN,..."


In [27]:
tweets.to_json('../dataset/tweet-processed.json', orient='index', default_handler=str)
tweets.to_excel('../dataset/tweet-label.xlsx')

In [35]:
# Moving the labelling from 5k to 16k
tweets_sample_df = pd.read_excel('../dataset/tweet-label-5000.xlsx')
tweets_sample_df

,Unnamed: 0,raw_text,emotion,is_spam
0,0,cepet bgt si…. #10YearsOf5SOS,s,NaN
1,1,Hallo Guys semoga kalian sehat selalu\n\nYang ...,NaN,NaN
2,2,#10YearsOf5SOS udah 10 tahun ajaa time flies.....,s,NaN
3,3,Ashton mi nene\n#BackTo2011 #10YearsOf5SOS #5...,NaN,NaN
4,4,"MYT mohegan sun, 2018 💖 #10YearsOf5SOS #5SOS h...",NaN,NaN
...,...,...,...,...
4124,5180,Xiao Zhan featured in ELLE: January 2021 issue...,NaN,NaN
4125,5181,xiao's eyeshadow. https://t.co/6HhFnOsQrg,NaN,NaN
4126,5182,Weibo TV series awards 2021 tutorial for male ...,NaN,NaN
4127,5183,wisp venti‘s first crush 🍃 #xiaoven #原神 #venti...,NaN,NaN


In [36]:
tweets_label_df = pd.read_excel('../dataset/tweet-label.xlsx')
tweets_label_df

,Unnamed: 0,raw_text,1st_clean_twitter,1st_clean_twitter_gensim,2nd_punctuation,2nd_punctuation_gensim,3rd_emoji,3rd_emoji_gensim,4th_tokenized,4th_tokenized_gensim,sentence,5_tagged
0,0,REPLY AND RT WITH THE TAGS 💜\n\n#AMAsTNT BTS @...,REPLY AND RT WITH THE TAGS 💜\n\n AMAsTNT BTS B...,REPLY AND RT WITH THE TAGS 💜\n\n BTS,REPLY AND RT WITH THE TAGS 💜\n\n AMAsTNT BTS B...,REPLY AND RT WITH THE TAGS 💜\n\n BTS,REPLY AND RT WITH THE TAGS \n\n AMAsTNT BTS BT...,REPLY AND RT WITH THE TAGS \n\n BTS,"['REPLY', 'AND', 'RT', 'WITH', 'THE', 'TAGS', ...","['REPLY', 'AND', 'RT', 'WITH', 'THE', 'TAGS', ...","Sentence: ""REPLY AND RT WITH THE TAGS AMAsTNT ...","['REPLY_PROPN', 'AND_PROPN', 'RT_PROPN', 'WITH..."
1,1,Lockscreens|@BTS_twt \n#BTSxAMAs #AMAsTNT #BTS...,Lockscreens|BTS_twt \n BTSxAMAs AMAsTNT BTS\...,Lockscreens| \n \nDon't repost! be honest!∅ \n,LockscreensBTStwt \n BTSxAMAs AMAsTNT BTS\nD...,Lockscreens \n \nDont repost! be honest!∅ \n,LockscreensBTStwt \n BTSxAMAs AMAsTNT BTS\nD...,Lockscreens \n \nDont repost! be honest!∅ \n,"['LockscreensBTStwt', 'BTSxAMAs', 'AMAsTNT', '...","['Lockscreens', 'Dont', 'repost', '!', 'be', '...","Sentence: ""LockscreensBTStwt BTSxAMAs AMAsTNT ...","['LockscreensBTStwt_PROPN', 'BTSxAMAs_PROPN', ..."
2,2,i love them 😭\n\n#AMAsTNT BTS @BTS_twt\n\nhttp...,i love them 😭\n\n AMAsTNT BTS BTS_twt\n\n,i love them 😭\n\n BTS \n\n,i love them 😭\n\n AMAsTNT BTS BTStwt\n\n,i love them 😭\n\n BTS \n\n,i love them \n\n AMAsTNT BTS BTStwt\n\n,i love them \n\n BTS \n\n,"['i', 'love', 'them', 'AMAsTNT', 'BTS', 'BTStwt']","['i', 'love', 'them', 'BTS']","Sentence: ""i love them AMAsTNT BTS BTStwt"" [...","['i_PROPN', 'love_PROPN', 'them_PROPN', 'AMAsT..."
3,3,[📸]\n\nNew Kids on The Block bersama BTS.\n\n...,[📸]\n\nNew Kids on The Block bersama BTS.\n\n...,[📸]\n\nNew Kids on The Block bersama BTS.\n\n...,📸\n\nNew Kids on The Block bersama BTS\n\nDon...,📸\n\nNew Kids on The Block bersama BTS.\n\nDo...,\n\nNew Kids on The Block bersama BTS\n\nDonn...,\n\nNew Kids on The Block bersama BTS.\n\nDon...,"['New', 'Kids', 'on', 'The', 'Block', 'bersama...","['New', 'Kids', 'on', 'The', 'Block', 'bersama...","Sentence: ""New Kids on The Block bersama BTS D...","['New_PROPN', 'Kids_PROPN', 'on_PROPN', 'The_P..."
4,4,[📸]\n\nShawn Stockman instagram update.\n\n📎 :...,[📸]\n\nShawn Stockman instagram update.\n\n📎 :...,[📸]\n\nShawn Stockman instagram update.\n\n📎 :...,📸\n\nShawn Stockman instagram update\n\n📎 \n...,📸\n\nShawn Stockman instagram update.\n\n📎 \...,\n\nShawn Stockman instagram update\n\n \n\n...,\n\nShawn Stockman instagram update.\n\n \n\...,"['Shawn', 'Stockman', 'instagram', 'update', '...","['Shawn', 'Stockman', 'instagram', 'update', '...","Sentence: ""Shawn Stockman instagram update BTS...","['Shawn_PROPN', 'Stockman_PROPN', 'instagram_P..."
...,...,...,...,...,...,...,...,...,...,...,...,...
16023,16023,Iiii mau liat cuplikan yang project seokjin ya...,Iiii mau liat cuplikan yang project seokjin ya...,Iiii mau liat cuplikan yang project seokjin ya...,Iiii mau liat cuplikan yang project seokjin ya...,Iiii mau liat cuplikan yang project seokjin ya...,Iiii mau liat cuplikan yang project seokjin ya...,Iiii mau liat cuplikan yang project seokjin ya...,"['Iiii', 'mau', 'liat', 'cuplikan', 'yang', 'p...","['Iiii', 'mau', 'liat', 'cuplikan', 'yang', 'p...","Sentence: ""Iiii mau liat cuplikan yang project...","['Iiii_PROPN', 'mau_AUX', 'liat_VERB', 'cuplik..."
16024,16024,SEOKJIN SAYANGKU 😭😭,SEOKJIN SAYANGKU 😭😭,SEOKJIN SAYANGKU 😭😭,SEOKJIN SAYANGKU 😭😭,SEOKJIN SAYANGKU 😭😭,SEOKJIN SAYANGKU,SEOKJIN SAYANGKU,"['SEOKJIN', 'SAYANGKU']","['SEOKJIN', 'SAYANGKU']","Sentence: ""SEOKJIN SAYANGKU"" [− Tokens: 2 −...","['SEOKJIN_NOUN', 'SAYANGKU_ADJ']"
16025,16025,HAPPY JIN DAY\nKIM SEOKJIN\n#LAHeartsJin #Moon...,HAPPY JIN DAY\nKIM SEOKJIN\n LAHeartsJin Moon...,HAPPY JIN DAY\nKIM SEOKJIN\n \n\n,HAPPY JIN DAY\nKIM SEOKJIN\n LAHeartsJin Moon...,HAPPY JIN DAY\nKIM SEOKJIN\n \n\n,HAPPY JIN DAY\nKIM SEOKJIN\n LAHeartsJin Moon...,HAPPY JIN D

In [41]:
tweets_merged_df = pd.merge(tweets_label_df, tweets_sample_df, how='left', on='raw_text')

In [42]:
tweets_merged_df.to_excel('../dataset/tweet-label-merged.xlsx')